# Imports

In [1]:
import Pkg; Pkg.add("FFTW")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("MatrixEquations")
using Pkg
Pkg.add("SpecialFunctions")
using Random
using SpecialFunctions


    Updating registry at `C:\Users\cefal\.julia\registries\General.toml`
   Resolving package versions...
   Installed AbstractFFTs ──── v1.5.0
   Installed FFTW_jll ──────── v3.3.11+0
   Installed oneTBB_jll ────── v2022.0.0+0
   Installed IntelOpenMP_jll ─ v2025.0.4+0
   Installed Reexport ──────── v1.2.2
   Installed FFTW ──────────── v1.8.1
   Installed MKL_jll ───────── v2025.0.1+1
    Updating `C:\Users\cefal\.julia\environments\v1.11\Project.toml`
  [7a1cc6ca] + FFTW v1.8.1
    Updating `C:\Users\cefal\.julia\environments\v1.11\Manifest.toml`
  [621f4979] + AbstractFFTs v1.5.0
  [7a1cc6ca] + FFTW v1.8.1
  [189a3867] + Reexport v1.2.2
  [f5851436] + FFTW_jll v3.3.11+0
  [1d5cc7b8] + IntelOpenMP_jll v2025.0.4+0
  [856f044c] + MKL_jll v2025.0.1+1
  [1317d2d5] + oneTBB_jll v2022.0.0+0
  [4af54fe1] + LazyArtifacts v1.11.0
  [37e2e46d] + LinearAlgebra v1.11.0
  [e66e0078] + CompilerSupportLibraries_jll v1.1.1+0
  [4536629a] + OpenBLAS_jll v0.3.27+1
  [8e850b90] + libblastrampoline_jll

# Setting up rr, zz, tt

In [13]:
# setting the N value

N = 5;
T = 10;

# points in t direction
# trigpts returns N equally spaced points on an interval -1 to 1
# range is kind of like linspace! the length

t = pi* range(-1,1, length=N+1)[1:end-1];
t = reshape(t,:,1)


# chebpts returns chebyshev points of the second kind
# generates N+1 cheb pts in the interval [-1, 1], so includes endpoints
# points are clustered near the endpoints

r = [cos((pi*k)/(N-1)) for k in (N-1):-1:0];
z = [cos((pi*k)/(N-1)) for k in (N-1):-1:0];


# going from N-1 to 0 ensures that endpoints are included bc when you 
#plug in (N-1) and 0 into the formula, you get the endpoints included -1 and 1 regardless of N
# cosine part is just a formula for cheb pts of second kind

display(t)
display(z)
display(r)

5×1 reshape(::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, 5, 1) with eltype Float64:
 -3.141592653589793
 -1.8849555921538759
 -0.6283185307179586
  0.6283185307179586
  1.8849555921538759

5-element Vector{Float64}:
 -1.0
 -0.7071067811865475
  6.123233995736766e-17
  0.7071067811865476
  1.0

5-element Vector{Float64}:
 -1.0
 -0.7071067811865475
  6.123233995736766e-17
  0.7071067811865476
  1.0

In [14]:
# same row vector vertical stacked
rr = [ri for ri in r, zi in z, ti in t]

# same column vector horizontal stacked
zz = [zi for ri in r, zi in z, ti in t]

# every entry in a matrix the same, but each layer has different value
tt = [ti for ri in r, zi in z, ti in t]

rr = dropdims(rr; dims=4)
tt = dropdims(tt; dims=4)
zz = dropdims(zz; dims=4)

display(rr)
display(zz)
display(tt)

5×5×5 Array{Float64, 3}:
[:, :, 1] =
 -1.0          -1.0          -1.0          -1.0          -1.0
 -0.707107     -0.707107     -0.707107     -0.707107     -0.707107
  6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17
  0.707107      0.707107      0.707107      0.707107      0.707107
  1.0           1.0           1.0           1.0           1.0

[:, :, 2] =
 -1.0          -1.0          -1.0          -1.0          -1.0
 -0.707107     -0.707107     -0.707107     -0.707107     -0.707107
  6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17
  0.707107      0.707107      0.707107      0.707107      0.707107
  1.0           1.0           1.0           1.0           1.0

[:, :, 3] =
 -1.0          -1.0          -1.0          -1.0          -1.0
 -0.707107     -0.707107     -0.707107     -0.707107     -0.707107
  6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17
  0.707107      0.707107      0.707107      0.707107      0.707107
  1.0          

5×5×5 Array{Float64, 3}:
[:, :, 1] =
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0

[:, :, 2] =
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0

[:, :, 3] =
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0

[:, :, 4] =
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0

[:, :, 5] =
 -1.0  -0.7

5×5×5 Array{Float64, 3}:
[:, :, 1] =
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159

[:, :, 2] =
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496

[:, :, 3] =
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319

[:, :, 4] =
 0.628319  0.628319  0.628319  0.628319  0.628319
 0.628319  0.628319  0.628319  0.628319  0.628319
 0.628319  0.628319  0.628319  0.628319  0.628319

# Create Matrices

In [15]:
# get the matrices we want: C02, D2, A, coF
using LinearAlgebra, SparseArrays

# C01: Convert Chebyshev-T (T) coefficients to Chebyshev-U (U)
C01 = spdiagm(0 => 0.5*ones(N), 2 => -0.5*ones(N-2))
C01[1,1] = 1.0
println("C01:")
display(C01)
# C12: Convert Chebyshev-U to C^2, the continuous function space
K_1 = 1.0 ./ (1:N)
K_2 = 1.0 ./ (3:N)
C12 = spdiagm(0 => K_1, 2 => -K_2)
println("C12:")
display(C12)
# C02: Convert Chebyshev-T to C^2 (It's C01 * C12)
C02 = C01 * C12
println("C02:")
display(C02)

# D1: Differentiate Chebyshev-T and convert to Chebyshev-U
D1 = spdiagm(1 => 1:N-1)
println("D1: ")
display(D1)
# D2: Differentiate Chebyshev-T twice and convert to C²
D2 = spdiagm(2 => 2 .* (2:N-1))
println("D2: ")
display(D2)
# D0: Differentiate Chebyshev-T once
D0 = C01 \ D1
println("D0: ")
display(D0)

# R0: Multiply Cheb-T by x
R0 = spdiagm(-1 => 0.5*ones(N-1), 1 => 0.5*ones(N-1))
R0[2,1] = 1.0
println("R0:")
display(R0)
# R2: Multiply in C² space
denom = 4:2:2*N+2
K_sub  = ((1:N) ./ denom)[1:N-1]
K_super = ((3:N+2) ./ denom[1:N])[2:N]
R2 = spdiagm(-1 => K_sub, 1 => K_super)
println("R2:")
display(R2)
# R1: Multiply in Cheb-U basis
R1 = C12 \ R2 * C12
println("R1:")
display(R1)

# A: Laplacian operator
A = R2 * R2 * D2 + R2 * C12 * D1
println("A:")
display(A)

# coF: Laplacian multiplier
coF = R2 * R2 * C02
println("coF:")
display(coF)

C01:


5×5 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
 1.0   ⋅   -0.5    ⋅     ⋅ 
  ⋅   0.5    ⋅   -0.5    ⋅ 
  ⋅    ⋅    0.5    ⋅   -0.5
  ⋅    ⋅     ⋅    0.5    ⋅ 
  ⋅    ⋅     ⋅     ⋅    0.5

C12:


5×5 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
 1.0   ⋅   -0.333333    ⋅      ⋅ 
  ⋅   0.5    ⋅        -0.25    ⋅ 
  ⋅    ⋅    0.333333    ⋅    -0.2
  ⋅    ⋅     ⋅         0.25    ⋅ 
  ⋅    ⋅     ⋅          ⋅     0.2

C02:


...

5×5 Matrix{Float64}:
 0.0  0.5  0.0  0.0   0.0
 0.5  0.0  0.5  0.0  -0.166667
 0.0  0.5  0.0  0.5   0.0
 0.0  0.0  0.5  0.0   0.166667
 0.0  0.0  0.0  0.5   0.0

A:


5×5 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
  ⋅    ⋅    1.33333    ⋅     2.66667
  ⋅   0.25   ⋅        2.625   ⋅ 
  ⋅    ⋅    0.666667   ⋅     3.73333
  ⋅    ⋅     ⋅        1.125   ⋅ 
  ⋅    ⋅     ⋅         ⋅     1.6

coF:


5×5 SparseMatrixCSC{Float64, Int64} with 12 stored entries:
 0.166667    ⋅       -0.0138889    ⋅         -0.0666667
  ⋅         0.09375    ⋅         -0.046875     ⋅ 
 0.0833333   ⋅        0.0305556    ⋅         -0.0433333
  ⋅         0.03125    ⋅          0.0260417    ⋅ 
  ⋅          ⋅        0.025        ⋅         -0.00666667

# Set Initial Condition

In [134]:
# finit(r, z, θ, t) = r .* sin.(θ) .+ z .* cos.(θ) # (This was a test function)

w54 = 18.98013387517992112;  # 4th root of the J-Bessel function of order 5
w33 = 13.0152007216984344;   # 3rd root of the J-Bessel function of order 5
q1 = 3;                      # Oscillation in z.
q2 = 2;   

u(r, z, th, t) = 10*exp(-((pi*q1)^2 + w33^2)*t) .* sin.(q1*pi*z) .* besselj.(3, w33 .* r) .* sin.(3*th) .+ exp(-((pi*q2)^2 + w54^2)*t) .* sin.(q2*pi*z) .* besselj.(5, w54 .* r) .* cos.(5*th);
finit(r,z,th,t) = u(r,z,th,0);

CFS = zeros(size(rr))

println("made it here")

# Evaluate initial condition on the full (r, z, θ) grid
for j in 1:N
    CFS[:, j, :] .= finit(rr[:,j,:], zz[:,j,:], tt[:,j,:], 0)
end

display(CFS);

made it here


5×5×5 Array{Float64, 3}:
[:, :, 1] =
 -4.4188e-32   -1.73899e-16  -6.94104e-32   1.73899e-16   4.4188e-32
  4.65351e-17   0.183136      7.30972e-17  -0.183136     -4.65351e-17
  1.42337e-77   1.44113e-62  -2.23583e-77  -1.44113e-62  -1.42337e-77
 -4.65351e-17  -0.183136     -7.30972e-17   0.183136      4.65351e-17
  4.4188e-32    1.73899e-16   6.94104e-32  -1.73899e-16  -4.4188e-32

[:, :, 2] =
 -1.04126e-31  -2.34585e-16   2.47398e-32   2.34585e-16   1.04126e-31
 -2.48636e-16  -0.115718      5.36751e-16   0.115718      2.48636e-16
 -2.27722e-62  -2.30563e-47   3.57705e-62   2.30563e-47   2.27722e-62
  2.48636e-16   0.115718     -5.36751e-16  -0.115718     -2.48636e-16
  1.04126e-31   2.34585e-16  -2.47398e-32  -2.34585e-16  -1.04126e-31

[:, :, 3] =
  5.27934e-32  -7.57074e-17  -2.21748e-31   7.57074e-17  -5.27934e-32
  5.24132e-16   0.666691     -6.7711e-16   -0.666691     -5.24132e-16
  3.68462e-62   3.73059e-47  -5.78778e-62  -3.73059e-47  -3.68462e-62
 -5.24132e-16  -0.666691     

# chebtech2.vals2coeffs and trigtech.vals2coeffs in Julia 

In [135]:
# replicate chebtech2.vals2coeffs and trigtech.vals2coeffs function
using FFTW
using LinearAlgebra

function chebv2c(vals)
    N = size(vals, 1)
    coeffs = zeros(size(vals))

    for col = 1:size(vals, 2)
        v = vals[:, col] # single column vector
        v_backwards = v[end-1:-1:2] # backwards
        v_ext = vcat(v, v_backwards) # mirrored but not the first at the end, so like 1 2 3 2
        c_ext = real(fft(v_ext)) # fft but only take real part
        c = c_ext[1:N] / (N-1) # normalizes the first half, N = first half size
        c[1] = c[1] / 2
        c[end] = c[end] / 2
        coeffs[:, col] = c
    end

    return coeffs

end



function trigv2c(vals)
    N = size(vals, 1)

    coeffs = (1/N)*fftshift(fft(vals, 1), 1)
    # display("coeffs below")
    # display(size(coeffs))
    # display(coeffs)

    if ( mod(N, 2) != 0 ) # N is odd
        even_odd_fix = (-1).^(-(N-1)/2:(N-1)/2)
    else # N is even
        even_odd_fix = (-1).^((-N/2):(N/2-1))
    end

    # display("evenoddfix size")
    # display(size(even_odd_fix))
    # display(even_odd_fix)

    coeffs = coeffs .* even_odd_fix

    return coeffs
end


trigv2c (generic function with 1 method)

# Testing above implementations

In [ ]:
# vals_cheb = [-1; -0.5; 0; 0.5; 1]
# display(vals_cheb)
# vals_cheb_updated = chebv2c(vals_cheb);
# display("vals_cheb updated")
# display(vals_cheb_updated)


# vals_trig = [-1 -0.5im 0 0.5im 1; -1im -0.5 0 0.5 1im; -1 -0.5 0im 0.5 1]
# display(vals_trig)
# vals_trig_updated = trigv2c(vals_trig)
# display("vals_trig updated")
# display(vals_trig_updated)

In [ ]:
# T = 1
# display(CFS)
# CFS_after = V2C_cyl(CFS,"rzt")
# display(CFS_after);
# # CFS(:,:,:,2:T)=zeros
# # println(CFS)

In [ ]:
# function V2C_cyl(vals, a)
#     # julia uses mapslices with dims=d, which works directly on a specific dimension so no need to manually reshape like in MATLAB

#     dim = ndims(vals)
#     sz = size(vals)
#     # println(dim)
#     # println(length(a))
    
#     # check that length(a) = # of dimensions
#     @assert length(a) == dim "Length of 'a' must match number of dimensions in vals."
#     #println("got here past assert")

#     # loop through dims, deciding which transform to apply
#     for d in 1:dim
#         # if t, FFT along dim
#         if a[d] == 't' || a[d] == 'T'
#             vals = mapslices(x -> fft(x) / length(x), vals; dims=(1,2))
#             #println("t done")
#         else
#             if a[d] == 'r'
#                 vals = mapslices(x -> dct(x, 2), vals; dims=(2,3))
#                 #println("r done")
#             else a[d] == 'z'
#                 vals = mapslices(x -> dct(x, 2), vals; dims=(1,3))
#                 #println("z done")
#             end            
#         end

#         # julia uses mapslices, which works directly on a specific dimension so no need to reshape and permute dimensions like in MATLAB
#     end

#     return vals
# end

# Running V2C_cyl

In [136]:
using Printf


function pretty_print_3Darray_matlab_style(A::Array; digits=4)
    println()

    iscomplexarray = eltype(A) <: Complex

    for k in 1:size(A, 3)
        slice = A[:, :, k]
        abs_slice = iscomplexarray ? abs.(slice) : abs.(real.(slice))
        maxval = maximum(abs_slice)
        
        println("(:,:, $k) =\n")

        if maxval == 0.0
            scale = 0
            scaled = slice
        else
            scale = floor(Int, log10(maxval))
            scaled = slice / (10.0^scale)
        end

        use_scale = maxval < 1e-3 || maxval > 1e4

        if use_scale
            println(@sprintf("   1.0e%+d *\n", scale))
        end

        for i in 1:size(slice, 1)
            for j in 1:size(slice, 2)
                val = use_scale ? scaled[i, j] : slice[i, j]
                if iscomplexarray
                    re = @sprintf("%7.*f", digits, real(val))
                    im = @sprintf("%7.*f", digits, imag(val))
                    sign_str = startswith(im, "-") ? "" : "+"
                    print(" $re $sign_str $im" * "i  ")
                else
                    print(@sprintf("%8.*f  ", digits, val))
                end
            end
            println()
        end
        println()
    end
end

pretty_print_3Darray_matlab_style (generic function with 1 method)

In [137]:
function V2C_cyl(vals, a)
    dim = length(a)
    SIZE = size(vals)

    a = collect(lowercase(a))

    for kk in 1:dim
        # first, "vectorize" to catch all columns at once.
        vals = reshape(vals, SIZE[kk], div(length(vals), SIZE[kk]))

        # apply the appropriate transform
        if a[kk] == 't'
            vals = trigv2c(vals)
        elseif a[kk] == '0'
            # do nothing
        else
            vals = chebv2c(vals)
        end

        # Reshape and permute back
        if dim != 1
            vals = reshape(vals, SIZE)
            vals = permutedims(vals, (2, 3, 1))
        end

        if a[kk] == 'r' || a[kk] == 'z'
            for j in 2:2:SIZE[3]
                vals[:, :, j] .= -vals[:, :, j]
            end
        end
        
        # println("CFS after transformation on dimension ", a[kk])
        # pretty_print_array_matlab_style(vals)
        
    end

    return vals

end


V2C_cyl (generic function with 1 method)

In [138]:
CFS_func = V2C_cyl(CFS, "rzt")
println("CFS now: ")
pretty_print_3Darray_matlab_style(CFS_func)

CFS now: 

(:,:, 1) =

 -0.0000  -0.0000i    0.0000 +  0.0000i    0.0000  -0.0000i    0.0000  -0.0000i   -0.0000 +  0.0000i  
 -0.0000  -0.0000i    0.0000  -0.1271i   -0.0000 +  0.0000i   -0.0000 +  0.1271i    0.0000 +  0.0000i  
  0.0000  -0.0000i    0.0000  -0.0000i   -0.0000 +  0.0000i   -0.0000 +  0.0000i   -0.0000 +  0.0000i  
  0.0000 +  0.0000i   -0.0000 +  0.1271i    0.0000  -0.0000i    0.0000  -0.1271i   -0.0000  -0.0000i  
  0.0000 +  0.0000i   -0.0000  -0.0000i   -0.0000 +  0.0000i   -0.0000 +  0.0000i    0.0000  -0.0000i  

(:,:, 2) =

   1.0e-17 *

  0.0000 +  0.1008i    0.0000 +  0.1346i   -0.0000  -0.0000i    0.0000  -0.1346i    0.0000  -0.1008i  
  0.0908  -0.3289i   -2.5308 +  1.8832i    0.0000  -0.0000i    2.5308  -1.1534i   -0.0908 +  0.3289i  
 -0.0000 +  0.0000i   -0.0000 +  0.0000i    0.0000  -0.0000i    0.0000  -0.0000i    0.0000  -0.0000i  
 -0.0908 +  0.3289i    2.2204  -1.9376i   -0.0000 +  0.0000i   -2.5308 +  2.3180i    0.0908  -0.3289i  
 -0.0000  -0.1008i 

## Visualizing Coefficients for Different N

In [ ]:
# using Plots

# N = (3, 5, 10, 20)
# T = 1

# for curr_N in N
#     # generate r, z, t
#     r = [cos((pi*k)/(curr_N-1)) for k in (curr_N-1):-1:0]
#     z = [cos((pi*k)/(curr_N-1)) for k in (curr_N-1):-1:0]
#     t = pi* range(-1,1, length=curr_N+1)[1:end-1]
#     t = reshape(t,:,1)

#     # generate rr, zz, tt
#     rr = [ri for ri in r, zi in z, ti in t]
#     zz = [zi for ri in r, zi in z, ti in t]
#     tt = [ti for ri in r, zi in z, ti in t]
#     rr = dropdims(rr; dims=4)
#     zz = dropdims(zz; dims=4) 
#     tt = dropdims(tt; dims=4)

#     # set initial condition (dummy test function for now)
#     finit(r, z, θ, t) = r .* sin.(θ) .+ z .* cos.(θ)
#     finit_test(r,z,θ) = finit(r,z,θ,0)
#     curr_CFS = zeros(size(rr))
#     for j in 1:curr_N
#         curr_CFS[:, j, :] = finit_test(rr[:,j,:],zz[:,j,:], tt[:,j,:])
#     end

#     # transform to coefficient space
#     curr_CFS = V2C_cyl(curr_CFS, "rzt")

#     # plot decay along each variable r, z, t
#     dims = size(curr_CFS)
#     vars = ["r", "z", "t"]

#     for (i, var) in enumerate(vars)
#         # move curr dimension to front and flatten the rest
#         permuted = permutedims(curr_CFS, (i, setdiff(1:3, i)...)) # e.g. if i = 2, setdiff(1:3, 2)... = (1, 3) so this is (2, 1, 3)
#         reshaped = reshape(permuted, dims[i], :) # flatten all dimensions except the i dim
#         decay = [norm(reshaped[k, :]) for k in 1:dims[i]] # for each row (AKA each coefficient for that term k), take the Euclidean norm of that row vector

#         # plot variable decay
#         plot(decay;
#             yscale = :log10,
#             marker = :circle,
#             xlabel = "Term index",
#             ylabel = "|Coefficient|",
#             label = "N = $curr_N",
#             title = "Decay along $var (N = $curr_N)")
#         display(current())  # display plots in loop
#     end

# end

# Plug in for first four time steps

In [139]:
using Printf

function pretty_print_4Darray_matlab_style(A::Array; digits=4)
    println()
    
    if ndims(A) != 4
        error("This function only works with 4D arrays.")
    end

    iscomplexarray = eltype(A) <: Complex

    for l in 1:size(A, 4)
        for k in 1:size(A, 3)
            slice = A[:, :, k, l]
            abs_slice = iscomplexarray ? abs.(slice) : abs.(real.(slice))
            maxval = maximum(abs_slice)
            
            println("(:,:,$k,$l) =\n")
            
            if maxval == 0.0
                scale = 0
                scaled = slice
            else
                scale = floor(Int, log10(maxval))
                scaled = slice / (10.0^scale)
            end
            
            use_scale = maxval < 1e-3 || maxval > 1e4
            
            if use_scale
                println(@sprintf("   1.0e%+d *\n", scale))
            end
            
            for i in 1:size(slice, 1)
                for j in 1:size(slice, 2)
                    val = use_scale ? scaled[i, j] : slice[i, j]
                    if iscomplexarray
                        re = @sprintf("%7.*f", digits, real(val))
                        im = @sprintf("%7.*f", digits, imag(val))
                        sign_str = startswith(im, "-") ? "" : "+"
                        print(" $re $sign_str $im" * "i  ")
                    else
                        print(@sprintf("%8.*f  ", digits, val))
                    end
                end
                println()
            end
            println()
        end
    end
end

pretty_print_4Darray_matlab_style (generic function with 1 method)

In [140]:
alph = 1e-9

CFS_4D = complex(zeros(Float64, size(CFS_func)..., T))  # Create a new 4D array full of zeros of size (5,5,5,5)
CFS_4D[:,:,:,1] = CFS_func # copy over CFS_func into the first 4th slice
#println(size(CFS_4D))
# display(CFS_4D)


Hist = Vector{Array{Float64,3}}(undef, 3)  # Equivalent to cell(3,1)

# Plug in exact solution for the first four time steps
for j in 1:3
    Hist[j] = Array{Float64,3}(undef, size(rr))  # Allocate space for each time slice
    for k in 1:size(rr, 2)  # Assuming k indexes the 2nd dimension
        Hist[j][:,k,:] = finit(rr[:,k,:], zz[:,k,:], tt[:,k,:], j * alph)
    end
    CFS_4D[:,:,:,j+1] = V2C_cyl(Hist[j], "rzt")
end

pretty_print_4Darray_matlab_style(CFS_4D)


(:,:,1,1) =

 -0.0000  -0.0000i    0.0000 +  0.0000i    0.0000  -0.0000i    0.0000  -0.0000i   -0.0000 +  0.0000i  
 -0.0000  -0.0000i    0.0000  -0.1271i   -0.0000 +  0.0000i   -0.0000 +  0.1271i    0.0000 +  0.0000i  
  0.0000  -0.0000i    0.0000  -0.0000i   -0.0000 +  0.0000i   -0.0000 +  0.0000i   -0.0000 +  0.0000i  
  0.0000 +  0.0000i   -0.0000 +  0.1271i    0.0000  -0.0000i    0.0000  -0.1271i   -0.0000  -0.0000i  
  0.0000 +  0.0000i   -0.0000  -0.0000i   -0.0000 +  0.0000i   -0.0000 +  0.0000i    0.0000  -0.0000i  

(:,:,2,1) =

   1.0e-17 *

  0.0000 +  0.1008i    0.0000 +  0.1346i   -0.0000  -0.0000i    0.0000  -0.1346i    0.0000  -0.1008i  
  0.0908  -0.3289i   -2.5308 +  1.8832i    0.0000  -0.0000i    2.5308  -1.1534i   -0.0908 +  0.3289i  
 -0.0000 +  0.0000i   -0.0000 +  0.0000i    0.0000  -0.0000i    0.0000  -0.0000i    0.0000  -0.0000i  
 -0.0908 +  0.3289i    2.2204  -1.9376i   -0.0000 +  0.0000i   -2.5308 +  2.3180i    0.0908  -0.3289i  
 -0.0000  -0.1008i   -0.000

# HeatSolver3D and ZeroDOF

In [141]:
using MatrixEquations

# change to Julia from MATLAB

function Heat3DSolver(cfs,Q,N,C02,A,D2,coF)

    # % Solves the screened Poisson equation, [I-Q*Del^2]u=CFS in coeff. space
    # % Homogeneous Dirichlet boundary conditions.
    
    # % Discretization size: N
    
    Fapprox = zeros(ComplexF64, N,N,N)
    
    # bc = [ (-1).^(0:N-1) > 0 ;  (-1).^(0:N-1) < 0 ]
    # bc = [ ((-1).^(0:N-1) .> 0) ; ((-1).^(0:N-1) .< 0) ]
    bc = Float64.(vcat(((-1).^(0:N-1) .> 0)', ((-1).^(0:N-1) .< 0)'))
    
    for k = -(N-1)//2 : (N-1)//2
        k_idx = Int(k + (N+1)//2)
        
        AA = coF - Q*(A - k^2*C02)
        BB = C02
        CC = -Q*coF
        DD = D2
        RHS = coF*cfs[:,:,k_idx]*C02'
        #%RHS = RHS(1:N-2, 1:N-2)
        
        (AA, RHS) = zeroDOF(AA, BB, RHS, bc, zeros(2,N))
        (BB, RHS) = zeroDOF(BB, AA, RHS', bc, zeros(2,N))
        RHS = RHS'
        (CC, RHS) = zeroDOF(CC, DD, RHS, bc, zeros(2,N))
        (DD, RHS) = zeroDOF(DD, CC, RHS', bc, zeros(2,N))
        RHS = RHS'
        
        AA = AA[1:end-2,3:end]
        BB = BB[1:end-2,3:end]
        CC = CC[1:end-2,3:end]
        DD = DD[1:end-2,3:end]
        RHS = RHS[1:end-2,1:end-2]
        
        X22 = gsylv(AA, BB, CC, DD, RHS )
        X12 = bc[1:2,1:2] \ (-bc[1:2,3:end]*X22)
        X21 = ( bc[1:2,1:2]' \ (-bc[1:2,3:end]*X22') )'
        X11 = bc[1:2,1:2] \ (-bc[1:2,3:end]*X21)
        X = [ X11 X12 ; X21 X22 ]
        Fapprox[:, :, k_idx] = X
    
    end

    return Fapprox
    
end
 


function zeroDOF(C1, C2, E, B, G)
    # %ZERODOF   Eliminate so degrees of freedom in the matrix equation can be
    # %removed.
    
    for ii = 1:size(B, 1) #% For each boundary condition, zero a column.
        for kk = 1:size(C1, 1)
            if ( abs(C1[kk,ii]) > 10*eps(Float64) )
                c = C1[kk, ii] #% Constant required to zero entry out.
                C1[kk,:] = C1[kk,:] - c*B[ii,:]
                E[kk,:] = E[kk,:] - ComplexF64(c) .* (reshape(G[ii,:], 1, :) * C2')[1,:]
            end
        end
    end

    return (C1, E)
    
end

zeroDOF (generic function with 1 method)

# Full time step

In [143]:
q = (12/25)*alph

for d = 5:T
    RHS = (48/25)*CFS_4D[:,:,:,d-1]-(36/25)*CFS_4D[:,:,:,d-2]+(16/25)*CFS_4D[:,:,:,d-3]-(3/25)*CFS_4D[:,:,:,d-4]
    CFS_4D[:,:,:,d] = Heat3DSolver(RHS,q,N,C02,A,D2,coF)
end

pretty_print_4Darray_matlab_style(CFS_4D)


(:,:,1,1) =

 -0.0000  -0.0000i    0.0000 +  0.0000i    0.0000  -0.0000i    0.0000  -0.0000i   -0.0000 +  0.0000i  
 -0.0000  -0.0000i    0.0000  -0.1271i   -0.0000 +  0.0000i   -0.0000 +  0.1271i    0.0000 +  0.0000i  
  0.0000  -0.0000i    0.0000  -0.0000i   -0.0000 +  0.0000i   -0.0000 +  0.0000i   -0.0000 +  0.0000i  
  0.0000 +  0.0000i   -0.0000 +  0.1271i    0.0000  -0.0000i    0.0000  -0.1271i   -0.0000  -0.0000i  
  0.0000 +  0.0000i   -0.0000  -0.0000i   -0.0000 +  0.0000i   -0.0000 +  0.0000i    0.0000  -0.0000i  

(:,:,2,1) =

   1.0e-17 *

  0.0000 +  0.1008i    0.0000 +  0.1346i   -0.0000  -0.0000i    0.0000  -0.1346i    0.0000  -0.1008i  
  0.0908  -0.3289i   -2.5308 +  1.8832i    0.0000  -0.0000i    2.5308  -1.1534i   -0.0908 +  0.3289i  
 -0.0000 +  0.0000i   -0.0000 +  0.0000i    0.0000  -0.0000i    0.0000  -0.0000i    0.0000  -0.0000i  
 -0.0908 +  0.3289i    2.2204  -1.9376i   -0.0000 +  0.0000i   -2.5308 +  2.3180i    0.0908  -0.3289i  
 -0.0000  -0.1008i   -0.000

## func2grid replication

In [10]:
function ndgrid(arrays::AbstractVector...)
    sizes = map(length, arrays)
    grids = ntuple(d -> begin
        shape = ones(Int, length(arrays))
        shape[d] = sizes[d]
        reshape(arrays[d], shape...) |> a -> repeat(a, sizes ./ shape)
    end, length(arrays))
    return grids
end

ndgrid (generic function with 1 method)

In [9]:
function func2grid(f, N, ord, coeff)

    # recreate trigpts and chebpts 

    t = pi* range(-1,1, length=N+1)[1:end-1];
    t = reshape(t,:,1)

    r = [cos((pi*k)/(N-1)) for k in (N-1):-1:0];

    # using julia version of linspace (range) and turning it into a column vector

    eq = range(-1, 1, N)
    collect(eq)

    # set up grid replicate cell() in MATLAB using Julia Array{}

    grid = Array{Any}(undef, length(ord), 1)

    # only thing that changed in loops was indexing () -> []

    for i in 1:length(ord)
        if ( ord[i] == 't' || ord[i] == 'T' )
            grid[i] = t;
        elseif ( ord[i] == 'e' || ord[i] == 'E' )
            grid[i] = eq;
        else
            grid[i] = r;
        end
    end

    GRID = ndgrid(grid...)
    X = f(GRID...)

    return

    X

end

func2grid (generic function with 1 method)